In [1]:
'''
    Mengimpor Pustaka yang kita butuhkan
'''
import cv2
import mediapipe as mp
from datetime import datetime

In [2]:

'''
    Config Mediapipe Solution
'''
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5)

'''
    Init Variabel yang dibutuhkan
    - normal_chin => posisi titik y dagu normal
    - variabel untuk kalibrasi threshold (antisipasi ketinggian kamera/ dagu berubah drastis)
        1. change_detected => mendeteksi apakah ada perubahan drastis terjadi?
        2. change_frame_count => menghitung frame setelah terdeteksi adanya perubahan drastis
    - looking_down_timer => untuk menghitung seberapa lama menunduk
    - required_looking_time => threshold waktu untuk menentukan dia mengantuk apa enggak (dari lamanya menunduk)
'''
normal_chin = None
change_detected = False
change_frame_count = 0
looking_down_timer = None
required_looking_time = 2
 
thresh_frame_count = 120
thresh_calibrate = 0.45
thresh_comp = 0.15 #kompensasi dagu y

In [3]:
'''
    Define Landmark yang kita butuhkan
'''
chosen_left_eye_idxs  = [362, 385, 387, 263, 373, 380]
chosen_right_eye_idxs = [33,  160, 158, 133, 153, 144]
chosen_mouth_idxs = [
    #upper right mouth
    82, 38, 72, 37,
    81, 41, 73, 39,
    80, 42, 74, 40,
    183,184, 185, 191,
    61, 76, 62, 78,

    #upper middle mouth
    14,  13, 0, 11,

    #upper left mouth
    312, 268, 302, 267,
    311, 271, 303, 269,
    310, 272, 304, 270,
    415, 407, 408, 409,
    291, 292, 308, 306,
    324,325, 307, 375,

    #botom right mouth
    146, 77,
    88, 89,90, 91,
    178, 179, 180, 181,
    87, 86, 85, 84,

    #botom middle mouth
    12, 15, 16, 17,

    #botom left mouth
    318, 319, 320, 321,
    402, 403, 404, 405,
    317, 316, 315, 314
    ]
nose_idxs = [1]
forehead_idxs = [8]
our_landmarks = []

for idxs in (chosen_left_eye_idxs, chosen_right_eye_idxs, chosen_mouth_idxs, nose_idxs, forehead_idxs):
    our_landmarks.extend(idxs)

In [4]:
def putText(frame, val, x, y, unity=""):
    if len(unity) == 0:
        cv2.putText(frame, f'{val}', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    else:
        cv2.putText(frame, f'{val:.2f} {unity}', (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
def diffTime(looking_down_timer):
    return (datetime.now() - looking_down_timer).total_seconds()

In [5]:
'''
    TAHAP 1
        - Baca Kamera
        - Baca setiap Frame
            1. Ubah BGR ke RGB 
            2. Terapkan face_mesh ke frame RGB
    
    TAHAP 2 (INIT)
        - Jika ada wajah terdeteksi, cek setiap titik landmarknya
        - Pada setiap titik landmarknya, cek
            if normal_chin masih None:
                maka isi nilainya dengan titik landmark dagu (x, y)
            else:
                perbarui nilai chin (koordinat dagu setiap frame)
    
    TAHAP 3 (CALIBRATE)
        - Cek apakah selisih titik y dagu (chin) dengan y dagu awal (normal_chin) itu lebih dari setengah dari y dagu awal (maka ada perubahan drastis)
            1. jika change detected masih false (berarti baru pertama masuk if) ubah change_frame_count 
            2. set nilai change_detected jadi True
        - Kalo tidak ada perubahan, maka reset nilai change frame count dan change detected seperti awal
        - Kalo change_detected = True (ada perubahan drastis)
            1. cek apakah frame count nya > 0 ? jika iya -= 1 Jika tidak perbarui threshold normal_chin dan ubah change_detected jadi False

    
    TAHAP 4 (DETERMINE)
        - Jika dagu pada setiap frame nilainya > range_normal_chin (ketinggian dagu y awal di tambah kompensasi) 
            1. Jalankan looking_down_timer
            2. Cek selisih antara thresh_looking_down_timer dan looking_down_timer jika lebih dari 2 detik (maka mengantuk)
        - Jika tidak maka reset looking_down_timer = None 
'''

# Todo => TAHAP 1
cap = cv2.VideoCapture(0) 

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)
    # Todo => END TAHAP 1
    
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Todo => TAHAP 2  (I N I T)
            if normal_chin is None:
                normal_chin = (face_landmarks.landmark[8].x, face_landmarks.landmark[8].y)
            chin = (face_landmarks.landmark[8].x, face_landmarks.landmark[8].y)
            # Todo => END TAHAP 2
            
            
            # Todo => TAHAP 3  (C A L I B R A T E)
            if abs(chin[1] - normal_chin[1]) > normal_chin[1] * thresh_calibrate:  
                if change_detected == False:
                    change_frame_count = thresh_frame_count
                change_detected = True
                print(f'Cam Changed\t{abs(chin[1] - normal_chin[1])}\t{normal_chin[1] * thresh_calibrate}')
            else:
                change_frame_count = thresh_frame_count
                change_detected = False
            
            # Perform calibration if change was detected
            if change_detected:
                if change_frame_count > 0:
                    change_frame_count -= 1
                    print(f'-===  frame_count: {change_frame_count}  ===-')
                else:
                    normal_chin = chin
                    change_detected = False
            # Todo => END TAHAP 3
            
            
            # Todo => TAHAP 4 (D E T E R M I N E)
            range_normal_chin = normal_chin[1] + (normal_chin[1] * thresh_comp)
            if chin[1] > range_normal_chin:
                if looking_down_timer is None:
                    looking_down_timer = datetime.now()
                status = "Looking down"
            else:
                status = "Not looking down"
                looking_down_timer = None

            if looking_down_timer is not None and change_detected == False:
                diff_time = diffTime(looking_down_timer)
                putText(frame, diff_time, 10, 80, 'seconds')
                
                if diff_time > required_looking_time:
                    putText(frame, "GO SLEEP NOW", 10, 130)
            # Todo => END TAHAP 4
            
            
            print(f'{normal_chin[1]}\t{chin[1]}\t{range_normal_chin}')
            putText(frame, status, 10, 30)


            # Draw landmarks on the frame
            filtered_landmarks = [face_landmarks.landmark[idx] for idx in our_landmarks]
            for landmark in filtered_landmarks:
                x, y = int(landmark.x * frame.shape[1]), int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

    cv2.imshow("Head Angle Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

0.24488523602485657	0.24488523602485657	0.28161802142858505
0.24488523602485657	0.2431558072566986	0.28161802142858505
0.24488523602485657	0.24388091266155243	0.28161802142858505
0.24488523602485657	0.2446780502796173	0.28161802142858505
0.24488523602485657	0.24509099125862122	0.28161802142858505
0.24488523602485657	0.24557626247406006	0.28161802142858505
0.24488523602485657	0.24659419059753418	0.28161802142858505
0.24488523602485657	0.2471674382686615	0.28161802142858505
0.24488523602485657	0.24719387292861938	0.28161802142858505
0.24488523602485657	0.24695821106433868	0.28161802142858505
0.24488523602485657	0.24678567051887512	0.28161802142858505
0.24488523602485657	0.24739301204681396	0.28161802142858505
0.24488523602485657	0.24781659245491028	0.28161802142858505
0.24488523602485657	0.24828720092773438	0.28161802142858505
0.24488523602485657	0.24937117099761963	0.28161802142858505
0.24488523602485657	0.24973666667938232	0.28161802142858505
0.24488523602485657	0.25035935640335083	0.2

In [6]:
cap.release()
cv2.destroyAllWindows()